In [23]:
import pandas as pd
# input data
coro_data = pd.read_csv('coro_data.csv')
coro_data

,Host,Virus,HostTaxID,VirusTaxID,HostNCBIResolved,VirusNCBIResolved,ICTVRatified,HostGenus,HostFamily,HostOrder,...,PMID,ReleaseYear,ReleaseMonth,ReleaseDay,CollectionYear,CollectionMonth,CollectionDay,AssocID,DatabaseDOI,NCBIAccession
0,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2005,8.0,31.0,NaN,NaN,NaN,NaN,NaN,DQ160294.1
1,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,10.0,15.0,2007.0,8.0,23.0,NaN,NaN,"HQ392470.1, HQ392471.1"
2,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,10.0,15.0,2008.0,1.0,22.0,NaN,NaN,HQ392472.1
3,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,2.0,8.0,2007.0,9.0,13.0,NaN,NaN,"GU553361.1, GU553362.1"
4,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,8.0,14.0,2007.0,7.0,17.0,NaN,NaN,HQ012367.1
5,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,8.0,14.0,2007.0,7.0,24.0,NaN,NaN,HQ012368.1
6,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,8.0,14.0,2007.0,9.0,11.0,NaN,NaN,HQ012369.1
7,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,8.0,14.0,2008.0,1.0,25.0,NaN,NaN,HQ012371.1
8,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2010,8.0,14.0,2008.0,1.0,3.0,NaN,NaN,HQ012370.1
9,NaN,alphacoronavirus 1,33554,693997,True,True,True,NaN,NaN,carnivora,...,NaN,2018,6.0,5.0,2014.0,11.0,26.0,NaN,NaN,MF457591.1


In [24]:
# get coroviridae data with NCBIAccession number
id_accession_data = coro_data[['VirusTaxID','NCBIAccession']]
# remove na
id_accession_data.dropna(inplace=True)

# construct a dictionary, the key of which is ID number and value is NCBIAccession
dic_ID_Accession = {}

for key,value in zip(id_accession_data['VirusTaxID'],id_accession_data['NCBIAccession']):
    process_value = []
    # split the accession number by comma
    for i in value.split(','):
        # remove space
        process_value.append(i.replace(" ",""))
    dic_ID_Accession[key] = dic_ID_Accession.get(key, []) + process_value


<ipython-input-24-890880d44c55>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_accession_data.dropna(inplace=True)


In [3]:
# construct two lists to conserve id and accession
IDs = []
Accessions = []

# construct a two columns data frame
for key, value in dic_ID_Accession.items():
    n_value_count = len(value)
    IDs = IDs + [key]*n_value_count
    Accessions = Accessions + value

coro_id_accession_csv_data = pd.DataFrame(IDs, Accessions)
coro_id_accession_csv_data = coro_id_accession_csv_data.reset_index() # reset the column name
coro_id_accession_csv_data.columns = ['Accession', 'VirusID']

In [4]:
# remove duplication
coro_id_accession_csv_data = coro_id_accession_csv_data.drop_duplicates(['Accession', 'VirusID'])
# conserve data into a excel table
coro_id_accession_csv_data.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv',index=None)

In [19]:
# input coroviridae data with VirusID and Accession
import pandas as pd
coro_id_accession = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/coro_id_accession.csv')

In [22]:
coro_id_accession[coro_id_accession['Accession'].isna()]

,Accession,VirusID
14097,NaN,2042696
17286,NaN,983929
19633,NaN,2042604
21380,NaN,1335626
21459,NaN,694001
22667,NaN,693999


In [16]:
coro_id_accession.loc[14097]

Accession        nan
VirusID      2042696
Name: 14097, dtype: object

In [6]:
# extract the number that represents the NCBI id of url
import re
number = []
accession_data = coro_id_accession["Accession"]

for i in accession_data:
    pattern = re.compile(r"^[0-9]\d+")
    number = number + re.findall(pattern, i)


In [7]:
# get the accession number through the number id
import requests
import re

def get_fasta_info(id):
    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key=59cbd6ff6b3ce5d506f0eeb69e4e09362409'.format(id)
    fasta_info = requests.get(fasta_url,timeout=(10,45)).text
    return fasta_info

number_accession = {}
failured_number_id = []

# using WHILE can improve the readablity of code
# the step is designed to convert GI numbers to Accession number, which consumes a lot of time
for i in range(12):
    for id in number[i*100:(i+1)*100]:
        try:
            fasta_info = get_fasta_info(id)
            pattern = re.compile(r'>(\w+\d+.\d)')
            match = re.match(pattern, fasta_info).group()
            number_accession[id] = match.strip('>')
        except:
            failured_number_id.append(id)
            print("The {} id numbers are failured to convert to accession".format(id))
            fasta_info = get_fasta_info(failured_number_id[0])
            pattern = re.compile(r'>(\w+\d+.\d)')
            match = re.match(pattern, fasta_info).group()
            number_accession[id] = match.strip('>')
            print("The {} id numbers are successed to convert to accession".format(id))
            failured_number_id.pop(0)


The 87244987 id numbers are failured to convert to accession
The 87244987 id numbers are successed to convert to accession
The 182702146 id numbers are failured to convert to accession
The 182702146 id numbers are successed to convert to accession
The 226316432 id numbers are failured to convert to accession
The 226316432 id numbers are successed to convert to accession


In [8]:
# check the length
len(number_accession.keys())

1195

In [16]:
# the step, conserving the data into a csv file, is unnecessary
numberID_accession_data = pd.DataFrame.from_dict(number_accession, orient='index')
numberID_accession_data = numberID_accession_data.reset_index()
numberID_accession_data.columns = ['GI', 'Accession']
numberID_accession_data.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/GI_Accession.csv',index=None)

In [5]:
numberID_accession_data = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/GI_Accession.csv')


,GI,Accession


In [18]:
# use dictary data format to accomplish the step is to replace GI by Accession
coro_id_accession = coro_id_accession.replace(number_accession)

In [19]:
# conserve data
coro_id_accession.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/final_Acce_VirusID.csv',index=None)